# A simple chatbot agent

## Prepare the environment

In [1]:
import sys
sys.path.append('..')

from autogen import ConversableAgent

# from inswitch.llm.model import get_openai_model_config
from inswitch.agent.basic import get_chat_agent, get_fixed_reply_agent

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


## Human proxy

In [2]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config=False, #No code executor
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety.
    is_termination_msg=lambda msg: "terminate" in msg["content"].lower(),
)

## some agents

In [3]:
chatbot_system_message = "You are a helpful assistant"

knowledge_asker = get_chat_agent(
    name = "knowledge-asker",
    system_message="You will get an intent from the stakeholders on how to manage the system."
        "You need to figure out what knowledge about the system do you need."
        "You should reply to describe what knowledge you need, and that's all. Do not make any"
        "assumptions or decision, as you haven't seen the knowledge."
)

with open('fill-kg-tri.txt', 'r') as file:
    content = file.read()

knowledge_provider = get_fixed_reply_agent(
    'knowledge_provider', 
    reply=content
)

docker_gen_agent = get_chat_agent(
    name = "docker_gen_agent",
    system_message="Based on the user intent and the knowledge you have learned"
        "about the system, if you believe that user needs to deploy a docker"
        "container on a machine, please generate a docker command to deploy the container."
        "Remember to set the DOCKER_HOST environment before, to make sure"
        "your local docker client works on the target machine"
        "One tip: the knowledge main mention 'container', but it actually means image"
)

moderator = get_fixed_reply_agent(
    name="moderator",
    reply = ""
)


/workspaces/inSwitch/playground/../inswitch/agent/basic.py:14: UserWarning: Using None to signal a default code_execution_config is deprecated. Use {} to use default or False to disable code execution.
  return ConversableAgent(


### Now we start


In [4]:
chat_result = moderator.initiate_chats(
    [
        {
            "recipient": human_proxy,
            "message": "what do you want?",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": knowledge_asker,
            "message": "here is the intent",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": knowledge_provider,
            "message": "",
            "max_turns": 1,
            "summary_method": "last_msg"
        },
        {
            "recipient": docker_gen_agent,
            "message": "you can see the intent and the knowledge in the context",
            "max_turns": 1,
            "summary_method": "last_msg"
        }
    ]
)



********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to human_proxy):

what do you want?

--------------------------------------------------------------------------------
human_proxy (to moderator):

I want to add service on machine MA-005934, to check the tool performace

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
moderator (to knowledge-asker):

here is the intent
Context: 
I want to add service on machine MA-005934, to check the tool performace

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
knowledge-asker (to moderator):

To fulfill this intent, I need 

## Excise
1. Try another topic to begin the dialog
2. Create another chatbot with a different initial setting, e.g., a poet who always reply with poems (how?)